In [7]:
#pip install mplfinance

### Import Libraries

In [82]:
#import plotly.graph_objects as go

#import pandas as pd
#from datetime import datetime
#import numpy as np 
#import dash
#import matplotlib.pyplot as plt
#import mplfinance as mpf
#from matplotlib.cm import ScalarMappable
#from matplotlib.colors import Normalize

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpl

import finnhub
import os

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
# To ignore the warnings 
warnings.filterwarnings("ignore")

#### Import Dataset (Source: Finviz)

In [91]:
# Load datasets
df_TSLA = pd.read_csv('Updated TSLA_Minute_Dataset.csv', parse_dates=['date-time'])
df_TSLA.head()

,Unnamed: 0,date-time,open,close,volume,short_sma,long_sma,buy_signal,sell_signal
0,0,2024-07-12 07:00:00,236.55,237.26,16633.0,NaN,NaN,False,False
1,1,2024-07-12 07:01:00,237.22,237.25,3127.0,NaN,NaN,False,False
2,2,2024-07-12 07:02:00,237.20,236.35,7759.0,NaN,NaN,False,False
3,3,2024-07-12 07:03:00,236.36,236.44,4788.0,NaN,NaN,False,False
4,4,2024-07-12 07:04:00,236.39,235.90,12058.0,236.64,NaN,False,False


In [92]:
# Drop the Useless column
df_minute = df_TSLA.drop(['Unnamed: 0', 'short_sma', 'long_sma', 'buy_signal', 'sell_signal'], axis=1)

# Print the first few rows and column names
print(df_minute.head())
print("\nColumn names:")
print(df_minute.columns)

            date-time    open   close   volume
0 2024-07-12 07:00:00  236.55  237.26  16633.0
1 2024-07-12 07:01:00  237.22  237.25   3127.0
2 2024-07-12 07:02:00  237.20  236.35   7759.0
3 2024-07-12 07:03:00  236.36  236.44   4788.0
4 2024-07-12 07:04:00  236.39  235.90  12058.0

Column names:
Index(['date-time', 'open', 'close', 'volume'], dtype='object')


#### Resample the dataset (Create 30m, 15m, 5m, 3m datasets)

In [93]:
df_minute['date-time'] = pd.to_datetime(df_minute['date-time'])
df_minute.set_index('date-time', inplace=True)
df_minute.head()

,open,close,volume
date-time,,,
2024-07-12 07:00:00,236.55,237.26,16633.0
2024-07-12 07:01:00,237.22,237.25,3127.0
2024-07-12 07:02:00,237.20,236.35,7759.0
2024-07-12 07:03:00,236.36,236.44,4788.0
2024-07-12 07:04:00,236.39,235.90,12058.0


In [96]:
def resample_ohlcv(df, timeframe):
    resampled = df.resample(timeframe)
    
    return pd.DataFrame({
        'Open': resampled['open'].first(),
        'High': resampled['close'].max(),  # Assuming 'close' is the highest price in the interval
        'Low': resampled['close'].min(),   # Assuming 'close' is the lowest price in the interval
        'Close': resampled['close'].last(),
        'Volume': resampled['volume'].sum(),
    }).reset_index()

In [98]:
df_3m = resample_ohlcv(df_minute, '3T')
df_5m = resample_ohlcv(df_minute, '5T')
df_15m = resample_ohlcv(df_minute, '15T')
df_30m = resample_ohlcv(df_minute, '30T')

In [99]:
# Display the first few rows of each dataset
print("\n3-minute data:")
print(df_3m.head())
print("\n5-minute data:")
print(df_5m.head())
print("\n15-minute data:")
print(df_15m.head())
print("\n30-minute data:")
print(df_30m.head())


3-minute data:
            date-time    Open    High     Low   Close   Volume
0 2024-07-12 07:00:00  236.55  237.26  236.35  236.35  27519.0
1 2024-07-12 07:03:00  236.36  236.44  235.90  235.93  22030.0
2 2024-07-12 07:06:00  235.94  235.97  235.80  235.87  11089.0
3 2024-07-12 07:09:00  235.91  236.10  235.82  236.10   9001.0
4 2024-07-12 07:12:00  236.15  236.96  236.37  236.96  10344.0

5-minute data:
            date-time    Open    High     Low   Close   Volume
0 2024-07-12 07:00:00  236.55  237.26  235.90  235.90  44365.0
1 2024-07-12 07:05:00  235.95  235.97  235.80  235.82  18262.0
2 2024-07-12 07:10:00  235.89  236.96  236.09  236.96  17356.0
3 2024-07-12 07:15:00  236.96  237.00  236.71  236.88   7022.0
4 2024-07-12 07:20:00  236.88  237.29  237.05  237.29   9891.0

15-minute data:
            date-time    Open    High     Low   Close    Volume
0 2024-07-12 07:00:00  236.55  237.26  235.80  236.96   79983.0
1 2024-07-12 07:15:00  236.96  237.45  236.71  237.20   31818.0
2 2

#### Create the customize candlestick chart

#### Create a function for flexible input

In [111]:
def create_advanced_candlestick_chart(df_larger_scale, df_smaller_scale, title="Stock Data - Larger Scale Candlesticks with Smaller Scale Bubbles"):
    # Ensure date-time columns are in datetime format
    df_larger_scale['date-time'] = pd.to_datetime(df_larger_scale['date-time'])
    df_smaller_scale['date-time'] = pd.to_datetime(df_smaller_scale['date-time'])

    # Create the figure
    fig = make_subplots(rows=1, cols=1)

    # Add candlestick trace for larger scale data
    fig.add_trace(go.Candlestick(
        x=df_larger_scale['date-time'],
        open=df_larger_scale['Open'],
        high=df_larger_scale['High'],
        low=df_larger_scale['Low'],
        close=df_larger_scale['Close'],
        name='Larger Scale Candlesticks'
    ))

    # Determine colors based on price movement
    colors = ['red' if close <= open else 'green' for close, open in zip(df_smaller_scale['Close'], df_smaller_scale['Open'])]

    # Add horizontal bubble chart for smaller scale data
    fig.add_trace(go.Scatter(
        x=df_smaller_scale['date-time'],
        y=df_smaller_scale['Close'],
        mode='markers',
        marker=dict(
            size=df_smaller_scale['Volume'],
            sizemode='area',
            sizeref=2.*max(df_smaller_scale['Volume'])/(40.**2),
            sizemin=4,
            color=colors,
            showscale=False
        ),
        name='Smaller Scale Data'
    ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date and Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False
    )

    return fig

#### Example of 30 minute to 3 minute

In [105]:
M30_to_M3_fig = fig = create_advanced_candlestick_chart(df_30m, df_3m, title="Stock Data - 30M Candlesticks with 3M-scale Bubbles")

# Display the chart
M30_to_M3_fig.show()

#### Example of 15 minute to 5 minute

In [112]:
M15_to_M5_fig = fig = create_advanced_candlestick_chart(df_15m, df_5m, title="Stock Data - 15 minute Candlesticks with 5M-scale Bubbles")

# Display the chart
M15_to_M5_fig.show()